# 3_Concordances

- adding concordance information to the pelic_key_forms dataframe
- creating a concordance csv for manual annotation

#### Sections of the notebook
- [Initial setup](#Initial-setup)
- [Concordancer format](#Concordancer-format)
- [Concordancer contents](#Concordancer-contents)
- [Concordances dataframe](#Concordances-dataframe)
- [Concordances csv](#Concordances-csv)

In [1]:
# Importing necessary modules
import pandas as pd
import pprint
import pickle as pkl
import operator
import csv
from more_itertools import unique_everseen

# Setting preferred notebook format
%pprint # Turn off pretty printing
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" # Shows all output, not just last item
pd.set_option('display.max_columns', 999) # Allow viewing of all columns

Pretty printing has been turned OFF


In [2]:
# Reading in pickle file
pelic_key_forms = pkl.load(open('pelic_key_forms.pkl', 'rb'))
pelic_key_forms.head()

,anon_id,text,level_id,native_language,gender,toks_nltk,toks_pos,lemmas,errors_in_text,len_errors_in_text,toks_corrected,lem_errors,len_lem_errors,lems_corrected,toks_pos_errors,len_toks_pos_errors,toks_pos_corrected,forms_in_text,len_forms
answer_id,,,,,,,,,,,,,,,,,,,
3,dk5,In my country we usually don't use tea bags. F...,4,Turkish,Female,"[In, my, country, we, usually, do, n't, use, t...","[(In, IN), (my, PRP$), (country, NN), (we, PRP...","[in, my, country, we, usually, do, n't, use, t...",[],0,"[In, my, country, we, usually, do, n't, use, t...",[],0,"[in, my, country, we, usually, do, n't, use, t...",[],0,"[(In, IN), (my, PR), (country, nn), (we, PR), ...","[(heat, nn)]",1
25,gc5,Last week I planned to go paintball match' but...,4,Turkish,Male,"[Last, week, I, planned, to, go, paintball, ma...","[(Last, JJ), (week, NN), (I, PRP), (planned, V...","[last, week, i, plan, to, go, paintball, match...",[],0,"[Last, week, I, planned, to, go, paintball, ma...",[],0,"[last, week, i, plan, to, go, paintball, match...",[],0,"[(last, jj), (week, nn), (I, PR), (planned, vv...","[(accepted, vv)]",1
30,er4,when you want to enjoy drinking a tea you have...,4,Arabic,Male,"[when, you, want, to, enjoy, drinking, a, tea,...","[(when, WRB), (you, PRP), (want, VBP), (to, TO...","[when, you, want, to, enjoy, drink, a, tea, yo...","[coract, finaly]",2,"[when, you, want, to, enjoy, drinking, a, tea,...","[coract, finaly]",2,"[when, you, want, to, enjoy, drink, a, tea, yo...","[(coract, JJ), (finaly, NN)]",2,"[(when, WR), (you, PR), (want, vv), (to, TO), ...","[(correct, jj)]",1
97,ea3,Here are a few instructions for typing in Engl...,4,Korean,Male,"[Here, are, a, few, instructions, for, typing,...","[(Here, RB), (are, VBP), (a, DT), (few, JJ), (...","[here, be, a, few, instruction, for, type, in,...",[],0,"[Here, are, a, few, instructions, for, typing,...",[],0,"[here, be, a, few, instruction, for, type, in,...",[],0,"[(here, rr), (are, vv), (a, DT), (few, jj), (i...","[(open, vv), (select, vv)]",2
111,cz7,Every one like a special kind of food. For me ...,4,Arabic,Male,"[Every, one, like, a, special, kind, of, food,...","[(Every, DT), (one, CD), (like, IN), (a, DT), ...","[every, one, like, a, special, kind, of, food,...",[],0,"[Every, one, like, a, special, kind, of, food,...",[],0,"[every, one, like, a, special, kind, of, food,...",[],0,"[(Every, DT), (one, CD), (like, IN), (a, DT), ...","[(special, jj), (specially, rr)]",2


### Concordancer format
Code in this section adapted from work (not public) by Dr Na-Rae Han.

In [3]:
# Creating a function for finding a specific word in a tokenized text
def concord_toks(text, word, num):
    padding = [('')] * num             # non-word padding, in the specified size
    padded = padding + text + padding   # pad in front and back
    outlist = []
    for i in range(len(padded)):
        if padded[i] == word:
            outlist.append((padded[i-num:i], padded[i], padded[i+1:i+1+num]))
    return outlist

In [4]:
# Creating a function for flattening the resulting list from the previous function
def flatten(outlist):
    flatlist = []
    for x in outlist:
        flatlist.append((' '.join(x[0]), x[1], ' '.join(x[2])))
    return flatlist

In [5]:
# Creating a function to improve the appearance of the ouput from the previous list 
# to look like a typical concordance with node words aligned in a column.
def prettify(flatlist):
    return ['{:>40} {:^12} {:<40}'.format(x,y,z) for (x,y,z) in flatlist]

In [6]:
# Creating an overall function combing the three sub functions above
def concordance(text, word, num):
    #return prettify(flatten(concord_toks(text, word, num))) 
    ### Above line can be used to go directly to prettify version, but then difficult to sort concordance lines alphabetically
    return flatten(concord_toks(text, word, num))

### Concordancer contents
The concordance column is based on the forms in 'forms_in_text' column.

In [7]:
# Creating a function for getting the concordances based on a tokenized text and a list of the node words
def get_concs(tok_text, forms_list):
    conclist = []
    for x in tok_text:
        if x in [x[0] for x in forms_list]:
            conclist.append(concordance(tok_text, x, 5))
    return [x for y in list(unique_everseen(conclist)) for x in y]

In [8]:
# Applying the above function to create a new column with the concordance lines for each text
pelic_key_forms['concordance'] = pelic_key_forms[['toks_corrected', 'forms_in_text']]\
.apply(lambda x: get_concs(x[0],x[1]), axis=1)

#### Creating an 'offset' column to be able to refer back to this data later, i.e. their index in the text.

In [9]:
# Create function to return only items which match 'forms_in_text'
def get_offset(tok_list, forms):
    new_list =  tok_list.copy()
    new_forms = forms.copy()
    return [x for x in list(enumerate(new_list)) if x[1] in [x[0] for x in new_forms]]

# Creating an offset column by applying the above function
pelic_key_forms['offset'] = pelic_key_forms[['toks_corrected','forms_in_text']]\
.apply(lambda x: get_offset(x[0],x[1]), axis=1)

In [10]:
# Repeating the above process but with the POS information as well

# Need new function to return only items which match 'forms_in_text' including POS
def get_offset2(tok_pos_list, forms_pos):
    new_list =  tok_pos_list.copy()
    new_forms = forms_pos.copy()
    new_list = list(enumerate(new_list))
    return [x for x in new_list if x[1] in new_forms]

# Creating an offset_POS column by applying the above function
pelic_key_forms['offset_POS'] = pelic_key_forms[['toks_pos_corrected','forms_in_text']]\
.apply(lambda x: get_offset2(x[0],x[1]), axis=1)

In [11]:
pelic_key_forms.head()

,anon_id,text,level_id,native_language,gender,toks_nltk,toks_pos,lemmas,errors_in_text,len_errors_in_text,toks_corrected,lem_errors,len_lem_errors,lems_corrected,toks_pos_errors,len_toks_pos_errors,toks_pos_corrected,forms_in_text,len_forms,concordance,offset,offset_POS
answer_id,,,,,,,,,,,,,,,,,,,,,,
3,dk5,In my country we usually don't use tea bags. F...,4,Turkish,Female,"[In, my, country, we, usually, do, n't, use, t...","[(In, IN), (my, PRP$), (country, NN), (we, PRP...","[in, my, country, we, usually, do, n't, use, t...",[],0,"[In, my, country, we, usually, do, n't, use, t...",[],0,"[in, my, country, we, usually, do, n't, use, t...",[],0,"[(In, IN), (my, PR), (country, nn), (we, PR), ...","[(heat, nn)]",1,"[(like 20 minutes with low, heat, for it to be...","[(44, heat)]","[(44, (heat, nn))]"
25,gc5,Last week I planned to go paintball match' but...,4,Turkish,Male,"[Last, week, I, planned, to, go, paintball, ma...","[(Last, JJ), (week, NN), (I, PRP), (planned, V...","[last, week, i, plan, to, go, paintball, match...",[],0,"[Last, week, I, planned, to, go, paintball, ma...",[],0,"[last, week, i, plan, to, go, paintball, match...",[],0,"[(last, jj), (week, nn), (I, PR), (planned, vv...","[(accepted, vv)]",1,"[(instead of me ? He, accepted, my order and h...","[(36, accepted)]","[(36, (accepted, vv))]"
30,er4,when you want to enjoy drinking a tea you have...,4,Arabic,Male,"[when, you, want, to, enjoy, drinking, a, tea,...","[(when, WRB), (you, PRP), (want, VBP), (to, TO...","[when, you, want, to, enjoy, drink, a, tea, yo...","[coract, finaly]",2,"[when, you, want, to, enjoy, drinking, a, tea,...","[coract, finaly]",2,"[when, you, want, to, enjoy, drink, a, tea, yo...","[(coract, JJ), (finaly, NN)]",2,"[(when, WR), (you, PR), (want, vv), (to, TO), ...","[(correct, jj)]",1,"[(you have to know the, correct, way to make i...","[(13, correct)]","[(13, (correct, jj))]"
97,ea3,Here are a few instructions for typing in Engl...,4,Korean,Male,"[Here, are, a, few, instructions, for, typing,...","[(Here, RB), (are, VBP), (a, DT), (few, JJ), (...","[here, be, a, few, instruction, for, type, in,...",[],0,"[Here, are, a, few, instructions, for, typing,...",[],0,"[here, be, a, few, instruction, for, type, in,...",[],0,"[(here, rr), (are, vv), (a, DT), (few, jj), (i...","[(open, vv), (select, vv)]",2,"[(Open the Format menu and, select, `` paragra...","[(105, select)]","[(100, (open, vv)), (105, (select, vv))]"
111,cz7,Every one like a special kind of food. For me ...,4,Arabic,Male,"[Every, one, like, a, special, kind, of, food,...","[(Every, DT), (one, CD), (like, IN), (a, DT), ...","[every, one, like, a, special, kind, of, food,...",[],0,"[Every, one, like, a, special, kind, of, food,...",[],0,"[every, one, like, a, special, kind, of, food,...",[],0,"[(Every, DT), (one, CD), (like, IN), (a, DT), ...","[(special, jj), (specially, rr)]",2,"[( Every one like a, special, kind of food . F...","[(4, special), (17, specially)]","[(4, (special, jj)), (17, (specially, rr))]"


In [12]:
# Doing a spot check that output is correct
%pprint # Pretty printing required to see if nodes align correctly
pelic_key_forms.iloc[50,19]
len(pelic_key_forms.iloc[50,19])
pelic_key_forms.iloc[50,20]
len(pelic_key_forms.iloc[50,20])

Pretty printing has been turned ON


[('I have mistakes I can', 'correct', 'them before I send this')]

1

[(427, 'correct')]

1

In [13]:
# The offset columns need to be sorted by x[1] then x[0] (first the word then the index) 
# to match the concordance column.
for x in pelic_key_forms['offset']:
    x.sort(key = operator.itemgetter(1))
    
for x in pelic_key_forms['offset_POS']:
    x.sort(key = operator.itemgetter(1))
    
# And then also sorting the concordances alphabetically by node word
pelic_key_forms['concordance'] = [sorted(x, key=lambda x: x[1]) for x in pelic_key_forms.concordance]

### Concordances dataframe

In [14]:
# Creating a new dataframe based on pelic_key_forms but with each concordance as a row (rather than each text)
conc_df = pelic_key_forms[['offset','offset_POS','concordance','level_id','native_language','gender','toks_pos_corrected']].reset_index()

In [15]:
# Creating a new column with tuples of the offset and the concordance (required later but easier to create here)
conc_df['offset_conc'] = list(zip(conc_df.offset_POS, conc_df.concordance))
conc_df['offset_conc'] = [list(zip(x[0],x[1])) for x in conc_df.offset_conc]

In [16]:
# Splitting this new column so that each item in the offset gets its own row (and doing some clean up)
conc_df = conc_df.offset_conc.apply(pd.Series) \
.merge(conc_df, left_index = True, right_index = True) \
.drop(['offset_conc'], axis = 1) \
.melt(id_vars = ['concordance', 'offset', 'offset_POS','answer_id','level_id', 'native_language', 'gender','toks_pos_corrected'], value_name = 'offset_conc') \
.drop("variable", axis = 1) \
.dropna() \

In [17]:
# The 'offset_conc' column is the target - each row is a unique offset and concordance

# Dropping columns for now that contain duplicate info due to the above process.
conc_df = conc_df.drop(['offset', 'offset_POS','concordance'], axis = 1)

# Re-splitting the new offset_conc back into two columns
conc_df['offset_POS'] = [x[0] for x in conc_df.offset_conc]
conc_df['conc'] = [x[1] for x in conc_df.offset_conc] # NOTE: commas are around the node word (not from Ss)
conc_df.head()

,answer_id,level_id,native_language,gender,toks_pos_corrected,offset_conc,offset_POS,conc
0,3,4,Turkish,Female,"[(In, IN), (my, PR), (country, nn), (we, PR), ...","((44, (heat, nn)), (like 20 minutes with low, ...","(44, (heat, nn))","(like 20 minutes with low, heat, for it to be ..."
1,25,4,Turkish,Male,"[(last, jj), (week, nn), (I, PR), (planned, vv...","((36, (accepted, vv)), (instead of me ? He, ac...","(36, (accepted, vv))","(instead of me ? He, accepted, my order and he..."
2,30,4,Arabic,Male,"[(when, WR), (you, PR), (want, vv), (to, TO), ...","((13, (correct, jj)), (you have to know the, c...","(13, (correct, jj))","(you have to know the, correct, way to make it..."
3,97,4,Korean,Male,"[(here, rr), (are, vv), (a, DT), (few, jj), (i...","((100, (open, vv)), (Open the Format menu and,...","(100, (open, vv))","(Open the Format menu and, select, `` paragrap..."
4,111,4,Arabic,Male,"[(Every, DT), (one, CD), (like, IN), (a, DT), ...","((4, (special, jj)), ( Every one like a, speci...","(4, (special, jj))","( Every one like a, special, kind of food . For)"


In [18]:
# Sorting dataframe by offset (word then offset number)
conc_df['offset_num'] = [x[0] for x in conc_df.offset_POS]
conc_df['offset_word_POS'] = [x[1] for x in conc_df.offset_POS]
conc_df = conc_df.sort_values(by = ['offset_word_POS', 'answer_id','offset_num']).reset_index(drop=True)

In [19]:
# Creating pickle of this dataframe for later use
conc_df.to_pickle("conc_df.pkl")

In [20]:
# Dropping duplicates - from manual checking, it was found that some texts have two versions back to back, 
# some might be repetitions of task prompts, and some are mislabelled as version 1.
conc_df = conc_df.drop_duplicates(subset='conc', keep="first")
conc_df.head()
len(conc_df)

,answer_id,level_id,native_language,gender,toks_pos_corrected,offset_conc,offset_POS,conc,offset_num,offset_word_POS
0,119,4,Japanese,Male,"[(It, PR), ('s, vv), (difficult, jj), (to, TO)...","((92, (accept, vv)), (much money , he can, acc...","(92, (accept, vv))","(much money , he can, accept, an advanced educ...",92,"(accept, vv)"
1,119,4,Japanese,Male,"[(It, PR), ('s, vv), (difficult, jj), (to, TO)...","((150, (accept, vv)), ('s difficult for him to...","(150, (accept, vv))","('s difficult for him to, accept, enough educa...",150,"(accept, vv)"
2,152,5,Korean,Female,"[(There, EX), (are, vv), (many, jj), (qualitie...","((204, (accept, vv)), (, but she pretended to,...","(204, (accept, vv))","(, but she pretended to, accept, my opinion . ...",204,"(accept, vv)"
3,203,4,Arabic,Male,"[(I, PR), (was, vv), (working, vv), (as, IN), ...","((62, (accept, vv)), (example , they did n't, ...","(62, (accept, vv))","(example , they did n't, accept, the checks fo...",62,"(accept, vv)"
4,818,4,Arabic,Male,"[(The, DT), (long, nn), (week, nn), (end, nn),...","((113, (accept, vv)), (you money with you and,...","(113, (accept, vv))","(you money with you and, accept, a lot of purc...",113,"(accept, vv)"


6896

In [21]:
# Adding a new randomized index to reduce annotator fatigue
conc_df = conc_df.reset_index(drop = True)
conc_df = conc_df.reset_index(drop = False)
conc_df = conc_df.sample(frac=1)
conc_df = conc_df.reset_index(drop = True)

In [22]:
# Cleaning up the dataframe for easier viewing
conc_df.rename(columns= {'offset_POS':'offset', 'offset_word_POS':'node','offset_num':'node_index',
                         'native_language': 'L1', 'level_id':'level','toks_pos_corrected':'toks_pos',
                         'index':'orig_index'}, inplace=True)
conc_df = conc_df[['orig_index','answer_id','level','L1','gender','node', 'node_index', 'offset', 
                   'conc', 'offset_conc','toks_pos']]

In [23]:
# Making all POS tags in toks_POS lower case - this is necessary in later workbooks
conc_df.toks_pos = conc_df.toks_pos.apply(lambda row:[(x[0],x[1].lower()) for x in row])

In [24]:
len(conc_df)
conc_df.head()

6896

,orig_index,answer_id,level,L1,gender,node,node_index,offset,conc,offset_conc,toks_pos
0,2883,23396,3,Korean,Male,"(excited, jj)",177,"(177, (excited, jj))","(n't have to be very, excited, themselves . Yo...","((177, (excited, jj)), (n't have to be very, e...","[(If, in), (you, pr), (are, vv), (a, dt), (per..."
1,4161,1558,5,Korean,Female,"(national, jj)",71,"(71, (national, jj))","(indicates the united world because, national,...","((71, (national, jj)), (indicates the united w...","[(cause, nn), (and, cc), (effect, nn), (essay,..."
2,3233,10984,4,Korean,Male,"(expectancy, nn)",140,"(140, (expectancy, nn))","(dog . And its life, expectancy, is 7 to 15 ye...","((140, (expectancy, nn)), (dog . And its life,...","[(today, nn), (,, ,), (many, jj), (people, nn)..."
3,2517,3944,4,Korean,Female,"(correcting, vv)",293,"(293, (correcting, vv))","(substance , qi , thus, correcting, many of th...","((293, (correcting, vv)), (substance , qi , th...","[(As, in), (The, dt), (world, nn), (develop, n..."
4,2368,28209,5,Turkish,Male,"(correct, jj)",317,"(317, (correct, jj))","(the time , following the, correct, steps , yo...","((317, (correct, jj)), (the time , following t...","[(having, vv), (a, dt), (good, jj), (job, nn),..."


In [25]:
# Checking the distribution of the different forms
forms_count = print(conc_df.node.value_counts().to_string())
forms_count

(special, jj)            737
(back, rr)               679
(continue, vv)           260
(national, jj)           189
(structure, nn)          184
(accept, vv)             176
(open, vv)               173
(open, jj)               147
(correct, jj)            142
(expect, vv)             139
(expected, vv)           127
(wide, jj)               124
(nation, nn)             118
(select, vv)             118
(equal, jj)              116
(heat, nn)               112
(back, nn)               105
(exciting, jj)           101
(accepted, vv)            92
(opened, vv)              89
(nations, nn)             85
(correct, vv)             83
(confused, vv)            75
(expectations, nn)        73
(competitive, jj)         71
(correctly, rr)           65
(widely, rr)              62
(construction, nn)        57
(compete, vv)             57
(unexpected, jj)          57
(excited, vv)             56
(expectation, nn)         56
(excited, jj)             53
(back, jj)                52
(continues, vv

In [26]:
# Create pickle of this dataframe for later use
conc_df.to_pickle("conc_df.pkl")

### Concordances csv

In [27]:
# Zipping these columns together to prepare the concordance csv
conc_csv = zip(conc_df.answer_id, conc_df.offset, conc_df.conc)
conc_csv = list(conc_csv)

In [28]:
# Running the prettify function to prepare the csv for printing
conc_csv = [(x[0], x[1], prettify([x[2]])) for x in conc_csv]

In [29]:
len(conc_csv)
conc_csv[:5]

6896

[(23396,
  (177, ('excited', 'jj')),
  ["                     n't have to be very   excited    themselves . You know that              "]),
 (1558,
  (71, ('national', 'jj')),
  ['      indicates the united world because   national   boundary has been destroyed by          ']),
 (10984,
  (140, ('expectancy', 'nn')),
  ['                      dog . And its life  expectancy  is 7 to 15 years                        ']),
 (3944,
  (293, ('correcting', 'vv')),
  ['                   substance , qi , thus  correcting  many of the imbalances .                ']),
 (28209,
  (317, ('correct', 'jj')),
  ['                the time , following the   correct    steps , you can get                     '])]

In [30]:
# Write out conc_csv to csv
with open('conc_csv.csv', 'w', ) as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    for answer_id, offset, conc in conc_csv:
        wr.writerow([answer_id, offset, conc])

139

136

140

139

137

...

135

137

139

134

137